### Intro to Folium 

![mapa](images/geo_portada.jpg)

Lo primerito será siempre leer la documentación ¿no? PLS :cobete: :músculo: :fueguito: 
- https://python-visualization.github.io/folium/

In [1]:
#!pip3 install folium

In [2]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [3]:
inicial_lat = 40.4143851
inicial_long = -3.6820241

### Primerito Mapa 

Creamos un mapa básico a raíz de unas coordenadas con un zoom inicial

In [4]:
map_1 = folium.Map(location = [inicial_lat,inicial_long], zoom_start = 15)
map_1

### Marker

Os dejo por aquí la doc concreta de los marker
- https://python-visualization.github.io/folium/modules.html?highlight=marker

Para añadir markers en un mapa, tendremos que "añadir" elementos a dicho mapa, por eso, vamos a añadirlos al mapa anterior llamado map_1. 

In [5]:
#Marker Parque de Dobby
parque = Marker(location=[inicial_lat,inicial_long], tooltip = "Parque de Dobby")
parque.add_to(map_1)
map_1

Añadimos otro Marker y vemos que le podemos cambiar el icono y el color de fondo y del icono.

In [6]:
#Ubicación de la casa de Dobby
lavapi_lat = 40.408808
lavapi_long = -3.7034796

Pero...wait ¿quién es Dobby?

![mapa](images/dobby.jpeg)

Para modificar los iconos ¿de dónde los sacamos?
- https://fontawesome.com/v4.7.0/icons/
- https://getbootstrap.com/docs/3.3/components/

In [7]:
#prefix = fa lo ponemos cuando usemos los iconos de fontawesome.
icono = Icon(color = "blue",
             prefix = "fa",
             icon = "paw",
             icon_color = "black",
             tooltip = "Casa de Dobby"
)
loc = [lavapi_lat, lavapi_long]

In [8]:
#guardamos el marker en una variable
marker_cueva = Marker(location = loc, icon = icono)

In [9]:
#añadimos el marker al mapa original donde ya teníamos el parque de Dobby guardado
marker_cueva.add_to(map_1)
map_1

### Markers, muchos markers, todos los markers a la vez 🚀

Para enseñar este ejemplo, utilizamos un dataset donde tenemos ya unas columnas de latitud y longitud 

In [10]:
df = pd.read_csv('data/bici_clean.csv')

In [11]:
df.head()

,fecha,dia,year,horario,festividad,tipo_accidente,lesividad,meteo,distrito,direccion,lon,lat
0,2019-01-01 14:00:00,2019-01-01 00:00:00,2019,Tarde,Festivo,Colision,Moderada,Despejado,SALAMANCA,CALLE CASTELLO / CALLE DON RAMON DE LA CRUZ Ma...,-3.681209,40.428990
1,2019-01-02 21:00:00,2019-01-02 00:00:00,2019,Noche,Laborable,Colision,Grave,Despejado,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALE...,-3.654527,40.466921
2,2019-01-03 14:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,CALLE FELIPE ALVAREZ 10 Madrid,-3.623423,40.381971
3,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939
4,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939


In [12]:
#Como hay muchos datos y estamos haciendo pruebas, vamos a quedarnos solo con los de 2019
df_2019 = df[(df.year == 2019)]

In [13]:
#¿Cuántas clasificaciones de festividad hay?
df_2019.festividad.unique()

array(['Festivo', 'Laborable', 'Vispera'], dtype=object)

In [14]:
map_2 = Map(location= [40.428990,-3.681209], zoom_start = 15)
map_2

Vamos a iterar sobre todas las filas del dataframe buscando qué hay dentro de cada fila en la columna. 
Pondremos un icono a los accidentes en función de la columna "festivo"
Pero tendremos que guardar las configuraciones y añadir el Marker al final y todo esto dentro de un bucle 🤭🤯
- Primero la config del tooltip
- Después el iconito en función del festivo
- Por último añadir cada marker

Keep Calm, lo vemos 🙃

In [15]:
#Bueno, recordad que lo primerito es crear un mapa base al que añadimos cosas
map_2 = Map(location=[40.428990,-3.681209],zoom_start=15)

In [16]:
for i,row in df_2019.iterrows():
    #popup distrito
    distrito = {
        "location" : [row["lat"], row["lon"]],
        "tooltip" : row["distrito"]
    }
    
    if row["festividad"] == "Festivo":
        icon = Icon( color = "green",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "black"
        )
    elif row["festividad"] == "Vispera":
        icon = Icon( color = "blue",
                    prefix = "fa",
                    icon = "home",
                    icon_color = "black"
        )
    else:
        icon = Icon( color = "red",
                    prefix = "fa",
                    icon = "briefcase",
                    icon_color = "black"
        )
    Marker (**distrito,icon = icon).add_to(map_2)

¿Es el mejor modo de visualizar esto? Hemos creado un mapa con infinitos Markers...mazo de markers...pero ¿Hay otro modo de visualizar esta información un poquito mejor?
Cuando tenemos muchos datos agrupados 🗺

### Mapa de Calor

Vamos a comprobar cuántas categorías hay en la columna de horario, para hacer un mapa de calor. Primero hacemos un mapa con el horario de mañana, y después añadimos el horario de tarde con el Layer Control. 
Allá vamos 🔥

In [17]:
df.horario.unique()

array(['Tarde', 'Noche', 'Mañana'], dtype=object)

In [18]:
map_3 = Map(location=[40.428990,-3.681209],zoom_start=15)

In [19]:
morning = df[df.horario=="Mañana"]
morning_group = folium.FeatureGroup(name = "Mañana")
HeatMap(data = morning[["lat","lon"]], radius = 15).add_to(morning_group)
morning_group.add_to(map_3)
map_3

In [20]:
tarde = df[df.horario == "Tarde"]
tarde.head()

,fecha,dia,year,horario,festividad,tipo_accidente,lesividad,meteo,distrito,direccion,lon,lat
0,2019-01-01 14:00:00,2019-01-01 00:00:00,2019,Tarde,Festivo,Colision,Moderada,Despejado,SALAMANCA,CALLE CASTELLO / CALLE DON RAMON DE LA CRUZ Ma...,-3.681209,40.428990
2,2019-01-03 14:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,CALLE FELIPE ALVAREZ 10 Madrid,-3.623423,40.381971
3,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939
4,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939
6,2019-01-04 17:00:00,2019-01-04 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,CHAMARTÍN,CALLE PRADILLO 26 Madrid,-3.673358,40.448681


In [21]:
tarde_group = folium.FeatureGroup (name = "Tarde")
HeatMap(data = tarde[["lat","lon"]], radius = 15).add_to(tarde_group)
tarde_group.add_to(map_3)
folium.LayerControl(collapsed = False).add_to(map_3)
map_3

In [23]:
#Mini ejercicio --> Crea un mapa y pon un marker o dos :) 

In [24]:
map_3.save('data/primerito_mapa')